In [1]:
import os
import cv2
import pandas as pd

C:\Users\estel\AppData\Local\Temp\ipykernel_9004\2092889490.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data_dir = 'C:/Users/estel/Documents/Python_Codes/breast_cancer/Dataset_BUSI_with_GT/'
save_dir_img = 'C:/Users/estel/Documents/Python_Codes/breast_cancer/preprocessed/images/'
save_dir_mask = 'C:/Users/estel/Documents/Python_Codes/breast_cancer/preprocessed/masks/'
save_csv_path = 'C:/Users/estel/Documents/Python_Codes/breast_cancer/preprocessed/informacoes.csv'
labels = ['benign', 'malignant', 'normal']
resize_dim = (300, 300)  # Dimensão para redimensionar as imagens e máscaras

In [3]:
label_names = []
label_codes = []
names = []
masks_combined = []

for i, label in enumerate(labels):
    # Pegar lista dos arquivos
    label_dir = os.path.join(data_dir, label)
       
    # Pegar a lista dos arquivos das imagens
    image_files = [file for file in os.listdir(label_dir) if file.endswith('.png')]
   
    # Ordenar a lista dos arquivos das imagens
    image_files = sorted(image_files)
    
    # Dicionário para armazenar máscaras temporariamente
    temp_masks = {}
    
    # Pegar o nome do arquivo separado para imagem e mascara
    for file in image_files:
        if '_mask' in file:
            base_name = file.split('_mask')[0]
            if base_name not in temp_masks:
                temp_masks[base_name] = []
            temp_masks[base_name].append(file)
        else:
            # Salvar as imagens em um novo diretório separado apenas para as imagens
            img = cv2.imread(os.path.join(label_dir, file))
            img_resized = cv2.resize(img, resize_dim)
            cv2.imwrite(os.path.join(save_dir_img, file), img_resized)
            
            names.append(file)
            label_names.append(label)
            label_codes.append(i)
    
    # Processar máscaras
    for base_name, mask_files in temp_masks.items():
        if len(mask_files) > 1:
            # Combinar máscaras se houver mais de uma
            combined_mask = None
            for mask_file in mask_files:
                mask_path = os.path.join(label_dir, mask_file)
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                if combined_mask is None:
                    combined_mask = mask
                else:
                    combined_mask = cv2.add(combined_mask, mask)
            combined_mask_resized = cv2.resize(combined_mask, resize_dim)
            combined_mask_path = os.path.join(save_dir_mask, f'{base_name}_combined_mask.png')
            cv2.imwrite(combined_mask_path, combined_mask_resized)
            masks_combined.append(f'{base_name}_combined_mask.png')
        else:
            # Copiar máscara única para o diretório de máscaras
            mask_file = mask_files[0]
            mask_path = os.path.join(label_dir, mask_file)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask_resized = cv2.resize(mask, resize_dim)
            combined_mask_path = os.path.join(save_dir_mask, mask_file)
            cv2.imwrite(combined_mask_path, mask_resized)
            masks_combined.append(mask_file)

In [4]:
# Criar um dataframe com as informações
df = pd.DataFrame({
    'label_names': label_names,
    'label_codes': label_codes,
    'names': names,
    'masks_combined': masks_combined
})

# Salvar o dataframe em um arquivo CSV
df.to_csv(save_csv_path, index=False)